In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
os.getenv("LANGCHAIN_PROJECT")

'AgenticAICourse'

In [3]:
# Set environment variables for OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

# Set environment variables for LangChain
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set environment variables for Groq
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "")

In [4]:
from langchain_community.document_loaders import PDFMinerLoader

file_path = "../2.1_Data_Ingestion_and_Splitting/syllabus.pdf"
loader = PDFMinerLoader(file_path)

loader

In [5]:
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")
docs[0]

Cannot set gray stroke color because /'P7' is an invalid float value
Cannot set gray non-stroke color because /'P7' is an invalid float value
Cannot set gray stroke color because /'P13' is an invalid float value
Cannot set gray non-stroke color because /'P13' is an invalid float value
Cannot set gray stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P8' is an invalid float value
Cannot set gray stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray stroke color because /'P10' is an invalid float value
Cannot set gray non-stroke color because /'P10' is an invalid float value
Cannot set gray stroke color because /'P11' is an invalid float value
Cannot set gray non-stroke color because /'P11' is an invalid float value
Cannot set gray stroke color because /'P12' is an invalid float value
Cannot set gray non-stroke color because /'P12' is an invalid float valu

Number of documents loaded: 1


Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-01-30T20:27:03+00:00', 'title': 'Ultimate Data Science & GenAI Bootcamp', 'moddate': '2025-01-30T20:26:59+00:00', 'keywords': 'DAGdmhcqnYw,BAEmsmap8Lg,0', 'author': 'monal singh', 'containsaigeneratedcontent': 'Yes', 'total_pages': 34, 'source': '../2.1_Data_Ingestion_and_Splitting/syllabus.pdf'}, page_content="PYTHON +\nSTATS\n\nMACHINE\nLEARNING\n\nDEEP\nLEARNING\n\nNATURAL LANGUAGE PROCESSING\n\nCOMPUTER VISION\n\nRETRIEVAL AUGUMENT GENERATION\n\nGENERATIVE AI\n\nVECTOR DB\n\x0cThis course is designed for aspiring data scientists, machine learning enthusiasts, and\nprofessionals looking to build expertise in Python programming, data analysis, machine learning,\nand deep learning. Whether you are just starting or have some experience, this comprehensive\ncourse will equip you with the skills needed to work with real-world datasets, apply machine\nlearning algorithms, and deploy AI solutions. By the end 

In [6]:
print(docs[0].page_content[:3000])  # See if multiple headings or pages show up here

PYTHON +
STATS

MACHINE
LEARNING

DEEP
LEARNING

NATURAL LANGUAGE PROCESSING

COMPUTER VISION

RETRIEVAL AUGUMENT GENERATION

GENERATIVE AI

VECTOR DB
This course is designed for aspiring data scientists, machine learning enthusiasts, and
professionals looking to build expertise in Python programming, data analysis, machine learning,
and deep learning. Whether you are just starting or have some experience, this comprehensive
course will equip you with the skills needed to work with real-world datasets, apply machine
learning algorithms, and deploy AI solutions. By the end of the course, you’ll have a solid
foundation in AI, a portfolio of end-to-end projects, and the confidence to tackle complex
challenges in data science and AI.

Learning Objectives

Master Python Programming: Understand Python fundamentals, including data types,

control structures, and object-oriented programming, to write efficient and reusable

code.

Handle Data with Pandas and NumPy: Acquire skills to manipulat

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Maximum number of characters per chunk
    chunk_overlap=200     # Overlap between chunks to preserve context
)

# Apply to your single-document list
split_docs = text_splitter.split_documents(docs)

# Check number of chunks and preview one
print(f"Number of chunks: {len(split_docs)}")
print("\nFirst chunk preview:\n", split_docs[0].page_content[:500])

Number of chunks: 42

First chunk preview:
 PYTHON +
STATS

MACHINE
LEARNING

DEEP
LEARNING

NATURAL LANGUAGE PROCESSING

COMPUTER VISION

RETRIEVAL AUGUMENT GENERATION

GENERATIVE AI

VECTOR DB
This course is designed for aspiring data scientists, machine learning enthusiasts, and
professionals looking to build expertise in Python programming, data analysis, machine learning,
and deep learning. Whether you are just starting or have some experience, this comprehensive
course will equip you with the skills needed to work with real-world d


In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
# Initialize embeddings
embeddings = OpenAIEmbeddings()

# Create a vector store from the split documents
vector_store = Chroma.from_documents(
    split_docs, embedding= embeddings,
    persist_directory="chroma_db/course_syllabus"  # Directory to store the vector database
)

In [9]:
print(f"Number of vectors stored: {vector_store._collection.count()}")

Number of vectors stored: 210


In [10]:
query = "What topics are covered in Week 3?"

# Use similarity search (semantic search)
results = vector_store.similarity_search(query, k=3)  # Top 3 similar chunks

# Print results
for i, doc in enumerate(results, 1):
    print(f"\nResult {i}:\n{doc.page_content}")


Result 1:
Topics

Trend Analysis and Segmentation

Sentiment and Quality Analysis

Recommendation and Forecasting

Analyzing Bike Sharing Trends,
Customer Segmentation and Effective
Cross-Selling

Analyzing Movie Reviews Sentiment,
Analyzing Wine Types and Quality

Analyzing Music Trends and
Recommendations, Forecasting Stock
and Commodity Prices

Ultimate Data Science & GenAI Bootcamp       Page  17
Module 12

Machine Learning Foundations and
Techniques

This module provides a comprehensive introduction to machine learning, covering key algorithms
and techniques. You’ll learn the differences between supervised and unsupervised learning, as
well as the core concepts of regression, classification, and clustering. The module introduces
model evaluation metrics like accuracy, precision, recall, and F1-score, giving you the foundation to
understand and implement machine learning models in real-world scenarios.

Topics

Introduction to Machine Learning

Linear Regression

Regularization T

In [11]:
from langchain import hub

#Load the RAG Prompt from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [12]:
print("Input variables:", prompt.input_variables)
print("Partial variables:", prompt.partial_variables)

Input variables: ['context', 'question']
Partial variables: {}


In [13]:
retriever = vector_store.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
rag_chain.invoke("What is the Course all about? What kind of Duration are we talking about? Also is it suitable for beginners?")

'The course, "Ultimate Data Science & GenAI Bootcamp," covers a comprehensive curriculum from Python programming basics to advanced topics in deep learning, NLP, and generative AI. It is designed to be completed over approximately 7 to 8 months, with an estimated commitment of 6 hours per week. The course is suitable for beginners, as no prerequisites are required.'

In [19]:
rag_chain.invoke("Can you tell me about the course structure? What are the topics covered in Week 3?")

'The course structure for the Ultimate Data Science & GenAI Bootcamp includes a module on Python Foundations. In Week 3, the topics covered are likely related to Python programming, including data structures, control flow, string manipulation, and functions, as these are part of the foundational topics listed in the context. However, the specific topics for Week 3 are not explicitly mentioned in the provided context.'

In [20]:
rag_chain.invoke("Who would be the instructors for this course and what are their qualifications?")

'The instructors for the course are Sourangshu Pal, a Senior Data Scientist; Krish Naik, a Chief AI Engineer; Monal Kumar, a Data Scientist; Sunny Savita, a GenAI Engineer; Mayank Aggrawal, a Senior ML Engineer; and Darius B., Head of Product.'